# Profile & Global Context Module

Context processors, profile queries, and global template variables migrated from `app.py`.

In [ ]:
# --- Profile and global context helpers ---
from flask import session, render_template
from contextlib import closing
import datetime

# Import get_db from the db module
import sys
from db import get_db

def inject_globals():
    return {"current_year": datetime.datetime.now().year}

def inject_global_context():
    try:
        with closing(get_db()) as db:
            cur = db.cursor()
            cur.execute("SELECT business_name, first_name FROM profile WHERE id = 1")
            profile_row = cur.fetchone()
            if profile_row:
                global_profile = {
                    "business_name": profile_row["business_name"] or "Your Service Provider",
                    "first_name": profile_row["first_name"] or ""
                }
            else:
                global_profile = {"business_name": "Your Service Provider", "first_name": ""}
    except Exception:
        global_profile = {"business_name": "Your Service Provider", "first_name": ""}
    return {"profile": global_profile, "current_year": datetime.datetime.now().year}

def get_public_provider_profile(request, provider_id):
    """
    Main function called by app.py for the /profile/<provider_id> route
    """
    with closing(get_db()) as db:
        cur = db.cursor()
        
        if provider_id == 0:
            # Admin/main profile
            cur.execute("""
                SELECT first_name, business_name, contact_email, phone, base_zip, 
                       address, about, profile_photo
                FROM profile 
                WHERE id = 1
            """)
            profile_row = cur.fetchone()
            
            if not profile_row:
                return render_template("404.html", title="Profile Not Found"), 404
            
            profile = {
                "id": 0,
                "first_name": profile_row["first_name"] or "",
                "business_name": profile_row["business_name"] or "Service Provider",
                "contact_email": profile_row["contact_email"] or "",
                "phone": profile_row["phone"] or "",
                "base_zip": profile_row["base_zip"] or "",
                "address": profile_row["address"] or "",
                "about": profile_row["about"] or "",
                "profile_photo": profile_row["profile_photo"] or ""
            }
        else:
            # Registered provider
            cur.execute("""
                SELECT id, first_name, business_name, phone, base_zip, address, about, 
                       profile_photo, linkedin_url, facebook_url, instagram_url, 
                       twitter_url, website_url, youtube_url
                FROM providers 
                WHERE id = ? AND active = 1
            """, (provider_id,))
            provider_row = cur.fetchone()
            
            if not provider_row:
                return render_template("404.html", title="Profile Not Found"), 404
            
            first_name = provider_row["first_name"] or ""
            business_name = provider_row["business_name"] or ""
            
            # Handle generic business names
            if business_name in ["", "Individual", "Service Provider"] or not business_name.strip():
                display_name = f"{first_name.capitalize()}'s Services" if first_name else "Service Provider"
            else:
                display_name = business_name
            
            profile = {
                "id": provider_row["id"],
                "first_name": first_name.capitalize() if first_name else "",
                "business_name": display_name,
                "contact_email": "",  # Don't expose provider emails publicly
                "phone": provider_row["phone"] or "",
                "base_zip": provider_row["base_zip"] or "",
                "address": provider_row["address"] or "",
                "about": provider_row["about"] or "",
                "profile_photo": provider_row["profile_photo"] or "",
                "linkedin_url": provider_row.get("linkedin_url", ""),
                "facebook_url": provider_row.get("facebook_url", ""),
                "instagram_url": provider_row.get("instagram_url", ""),
                "twitter_url": provider_row.get("twitter_url", ""),
                "website_url": provider_row.get("website_url", ""),
                "youtube_url": provider_row.get("youtube_url", "")
            }
        
        # Get provider's services
        cur.execute("""
            SELECT id, title, description, price, is_certified, certification_proof
            FROM services
            WHERE provider_id = ? AND active = 1
            ORDER BY created_at DESC
        """, (provider_id,))
        services = cur.fetchall()
        
        # Get provider's products
        cur.execute("""
            SELECT id, title, description, price, image1, image2, image3, image4, image5
            FROM products
            WHERE provider_id = ? AND active = 1
            ORDER BY created_at DESC
        """, (provider_id,))
        products_raw = cur.fetchall()
        products = [dict(row) for row in products_raw]
        
        # Get provider's service areas
        cur.execute("""
            SELECT id, zip_code, radius_miles, created_at
            FROM provider_zips
            WHERE provider_id = ?
            ORDER BY created_at DESC
        """, (provider_id,))
        service_areas = cur.fetchall()
        
        # Get provider's upcoming events
        cur.execute("""
            SELECT id, title, description, date, location, zip, created_at
            FROM events
            WHERE provider_id = ? AND date >= date('now') AND status != 'rejected'
            ORDER BY date ASC
            LIMIT 5
        """, (provider_id,))
        upcoming_events = cur.fetchall()
    
    return render_template(
        "provider_public_profile.html",
        profile=profile,
        services=services,
        products=products,
        service_areas=service_areas,
        upcoming_events=upcoming_events,
        title=f"{profile['business_name']} - Profile"
    )